In [41]:
import requests
from urllib.request import urlopen
from lxml import etree
import re
import pandas as pd

dialogue_tmp = 'https://www.rev.com/blog/transcript-tag/donald-trump-interview-transcripts/page/{}'
quotes_tmp = 'https://www.goodreads.com/author/quotes/676.Donald_J_Trump?page={}'

In [21]:
dialogue_list = []
for i in range(1,4):
  url = dialogue_tmp.format(i)
  response = urlopen(url)
  htmlparser = etree.HTMLParser()
  tree = etree.parse(response, htmlparser)
  links = tree.xpath('//div[@class="fl-post-column"]//a/@href')
  for link in links:
    response = urlopen(link)
    htmlparser = etree.HTMLParser()
    tree = etree.parse(response, htmlparser)
    dialogue = tree.xpath('//div[@class="fl-callout-text"]/p//text()')
    dialogue_list.append(dialogue)


In [27]:
quotes_list = []
for i in range(1,12):
  url = quotes_tmp.format(i)
  response = urlopen(url)
  htmlparser = etree.HTMLParser()
  tree = etree.parse(response, htmlparser)
  text_list = re.findall(r'“(.+)”', ' '.join(tree.xpath('//div[@class="quoteText"]/text()')))
  quotes_list += text_list

In [98]:
history = []
for j, dialogue in enumerate(dialogue_list):
  for i, kek in enumerate(dialogue):
    if re.match(r"(.+): \(", kek):
      name = re.findall(r"(.+): \(", kek)[0]
    elif re.match(r"\n(.+)", kek):
      text = re.findall(r"\n(.+)", kek)[0]
      history.append([name, text])

In [102]:
dialogue_df = pd.DataFrame(history, columns=['name', 'text'])

In [104]:
quotes_df = pd.DataFrame(quotes_list, columns=['quote'])

In [105]:
dialogue_df.to_csv('dialogue_df.csv', index=False)
quotes_df.to_csv('quotes_df.csv', index=False)